In [48]:
import numpy as np 
import os
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import KFold
from collections import defaultdict
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV
import statistics
from sklearn.metrics import make_scorer, accuracy_score, roc_auc_score 
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from collections import Counter

In [49]:
filenames = os.listdir('./searched_songs/')
filepath = "./searched_songs/"

In [ ]:
threshold = 10

In [50]:
sum_rfaccuracy=0
count=0
lowest_accuracy = 100
# whole_dataset = pd.read_csv(filepath+"us.csv", encoding='utf-8',)
whole_dataset = pd.read_csv("./kaggle_dataset/song_data.csv")

In [51]:
whole_dataset.head()
# whole_dataset = whole_dataset.drop("Unnamed:0")


,song_name,song_popularity,song_duration_ms,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,audio_mode,speechiness,tempo,time_signature,audio_valence
0,Boulevard of Broken Dreams,73,262333,0.005520,0.496,0.682,0.000029,8,0.0589,-4.095,1,0.0294,167.060,4,0.474
1,In The End,66,216933,0.010300,0.542,0.853,0.000000,3,0.1080,-6.407,0,0.0498,105.256,4,0.370
2,Seven Nation Army,76,231733,0.008170,0.737,0.463,0.447000,0,0.2550,-7.828,1,0.0792,123.881,4,0.324
3,By The Way,74,216933,0.026400,0.451,0.970,0.003550,0,0.1020,-4.938,1,0.1070,122.444,4,0.198
4,How You Remind Me,56,223826,0.000954,0.447,0.766,0.000000,10,0.1130,-5.065,1,0.0313,172.011,4,0.574


In [52]:
# df.loc[:, ~df.columns.str.match('Unnamed')]
whole_dataset = whole_dataset.loc[:, ~whole_dataset.columns.str.match('Unnamed')]

In [53]:
whole_dataset.head()
print(len(whole_dataset))

18835


In [54]:
def detect_outliers(df, features):
    outlier_indices = []

    for c in features:
        # 1st quartile
        Q1 = np.percentile(df[c], 25)
        # 3rd quartile
        Q3 = np.percentile(df[c], 75)
        # IQR
        IQR = Q3 - Q1
        # Outlier step
        outlier_step = IQR * 1.5
        # detect outlier and their indeces
        outlier_list_col = df[(df[c] < Q1 - outlier_step)
                              | (df[c] > Q3 + outlier_step)].index  # filtre
        # store indeces
        # The extend() extends the list by adding all items of a list (passed as an argument) to the end.
        outlier_indices.extend(outlier_list_col)

    outlier_indices = Counter(outlier_indices)
    multiple_outliers = list(i for i, v in outlier_indices.items() if v > 2)

    return multiple_outliers



In [55]:
# outlier_features = ["popularity","length","danceability","energy","instrumentalness","liveness","loudness","speechiness","valence"]
outlier_features = ["song_popularity","song_duration_ms","danceability","energy","instrumentalness","liveness","loudness","speechiness","audio_valence"]
whole_dataset.loc[detect_outliers(whole_dataset,outlier_features)]
print(len(whole_dataset.loc[detect_outliers(whole_dataset,outlier_features)]))

325


In [56]:
whole_dataset = whole_dataset.drop(detect_outliers(whole_dataset, outlier_features),axis = 0).reset_index(drop = True)

In [57]:
print(len(whole_dataset))

18510


In [58]:
# threshold = whole_dataset["popularity"].quantile(0.70)

# df[df.a < df.a.quantile(.95)]

# threshold_df = whole_dataset.nlargest(int(len(whole_dataset.index)*0.60),'popularity')
# threshold=66.5

whole_dataset["popularity"]= [ 1 if i>=40 else 0 for i in whole_dataset.song_popularity ]
whole_dataset["popularity"].value_counts()
# print(threshold_df.head())

# print(len(threshold_df))

# threshold = np.mean(threshold_df.popularity)
# print(f"type of threshold {type(threshold)}")
# threshold=80
# print(f"threshold {threshold}")
# whole_dataset.loc[whole_dataset['popularity'] < threshold, 'thresholded_popularity'] = 0
# whole_dataset.loc[whole_dataset['popularity']
#                   >= threshold, 'thresholded_popularity'] = 1
# print(whole_dataset.thresholded_popularity.value_counts())

1    14082
0     4428
Name: popularity, dtype: int64

In [59]:
def change_type(var):
    whole_dataset[var] = whole_dataset[var].astype(int)

In [60]:
# column= ["key","length","mode"]
# for i in column:
#     change_type(i)



In [61]:
# x,y = whole_dataset[:] , whole_dataset.loc[:,'popularity']


whole_dataset.drop(["song_popularity","song_name"],axis=1,inplace=True)
x,y = whole_dataset.loc[:,whole_dataset.columns != 'popularity'], whole_dataset.loc[:,'popularity']
# x = (x - np.min(x))/(np.max(x)-np.min(x)).values
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state=42)
y=y.astype(int)
y_train= y_train.astype(int)
y_test= y_test.astype(int)
x_train.head()

,song_duration_ms,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,audio_mode,speechiness,tempo,time_signature,audio_valence
6225,287306,0.648,0.450,0.658,0.000003,2,0.264,-7.064,1,0.0472,106.799,4,0.781
6412,208546,0.139,0.708,0.436,0.000001,7,0.085,-7.004,1,0.0300,129.910,4,0.609
7882,130455,0.124,0.833,0.730,0.000003,6,0.875,-7.837,0,0.3490,140.063,4,0.499
6765,266533,0.309,0.337,0.717,0.000000,2,0.418,-5.772,1,0.1650,84.729,4,0.444
10062,183902,0.821,0.556,0.175,0.000108,4,0.120,-18.346,0,0.0404,92.067,4,0.501


In [62]:
whole_dataset.columns[whole_dataset.isnull().any()]

# song_data.columns[song_data.isnull().any()]
whole_dataset.isnull().sum()



song_duration_ms    0
acousticness        0
danceability        0
energy              0
instrumentalness    0
key                 0
liveness            0
loudness            0
audio_mode          0
speechiness         0
tempo               0
time_signature      0
audio_valence       0
popularity          0
dtype: int64

In [63]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(n_estimators=150,random_state = 3)
rf.fit(x_train,y_train)
print("Train ccuracy of random forest",rf.score(x_train,y_train))
print("Test accuracy of random forest",rf.score(x_test,y_test))
RandomForestClassifier_score=rf.score(x_test,y_test)
y_pred=rf.predict(x_test)
t_true=y_test

Train ccuracy of random forest 0.9963533225283631
Test accuracy of random forest 0.7982171799027553


In [64]:
cm = confusion_matrix(y_test, y_pred)
print('Confusion matrix: \n', cm)

Confusion matrix: 
 [[ 163  682]
 [  65 2792]]


In [65]:
print('Classification report: \n',
        classification_report(y_test, y_pred))

Classification report: 
               precision    recall  f1-score   support

           0       0.71      0.19      0.30       845
           1       0.80      0.98      0.88      2857

    accuracy                           0.80      3702
   macro avg       0.76      0.59      0.59      3702
weighted avg       0.78      0.80      0.75      3702

